<a href="https://colab.research.google.com/github/rer10013/ML-project/blob/main/Pytorch_Basic_3.2.5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# library
from sklearn import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
# import dataset - revised
dataset = datasets.load_breast_cancer()

X, y = dataset['data'], dataset['target']

In [4]:
# Tensor-ize
X = torch.FloatTensor(X)
y = torch.FloatTensor(y).view(-1, 1)

# reqularization
X = (X - X.mean(dim=0)) / X.std(dim=0)

# dataset maker
dset = TensorDataset(X, y)
loader = DataLoader(dset, batch_size=256, shuffle=True)

In [5]:
# model
class NeuralNetwork(nn.Module):
  def __init__(self, num_features):
    super().__init__()
    self.linear1 = nn.Linear(num_features, 4)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(4, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, X):
    out = self.linear1(X)
    out = self.relu(out)
    out = self.linear2(out)
    out = self.sigmoid(out)
    return out

model = NeuralNetwork(30)

In [6]:
# BCE
criterion = nn.BCELoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [7]:
# train model
def train(model, criterion, optimizer, loader):
  epoch_loss = 0

  for X_batch, y_batch in loader:
    optimizer.zero_grad()
    hypothesis = model(X_batch)
    loss = criterion(hypothesis, y_batch)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()

  return epoch_loss / len(loader)

In [8]:
# train
n_epochs = 100
for epoch in range(1, n_epochs + 1):
  loss = train(model, criterion, optimizer, loader)
  if epoch % 10 == 0:
    print(f'Epoch: {epoch}, Loss: {loss}')

Epoch: 10, Loss: 0.26706311106681824
Epoch: 20, Loss: 0.1546394924322764
Epoch: 30, Loss: 0.09831816454728444
Epoch: 40, Loss: 0.09093216309944789
Epoch: 50, Loss: 0.08932167043288548
Epoch: 60, Loss: 0.06856339424848557
Epoch: 70, Loss: 0.0669243795176347
Epoch: 80, Loss: 0.05962227036555608
Epoch: 90, Loss: 0.05024047568440437
Epoch: 100, Loss: 0.055282898247241974


In [9]:
# inference
y_precdicted1 = (model(X) >= 0.5).float()

# accuracy
score_of_predicted_model = (y_precdicted1 == y).float().mean()
print(f'Accuracy: {score_of_predicted_model}')

Accuracy: 0.9859402179718018


In [12]:
# save
torch.save(model.state_dict(), './model.pt')

# create new model & reload
loared_model = NeuralNetwork(30)
loared_model.load_state_dict(torch.load('./model.pt'))

# inference
y_precdicted2 = (loared_model(X) >= 0.5).float()

# accuracy
score_of_predicted_model = (y_precdicted2 == y).float().mean()
print(f'Accuracy: {score_of_predicted_model}')

Accuracy: 0.9859402179718018


<ipython-input-12-748a422c0d0c>:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loared_model.load_state_dict(torch.load('./model.pt'))
